In [ ]:
"""
Code done by,
  Pratik Shetty AI21MTECH12005
  Varshita Sharma AI21MTECH14009
  Sunamdha Harini AI21MTECH14002
  
"""

In [ ]:
!pip install transformers
!pip install tensorboard

In [1]:
import transformers as tf
tf.__version__

'4.18.0'

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
from transformers import BertModel, BertTokenizer
import torch

In [12]:
import pandas as pd
# df=pd.read_excel('/content/drive/My Drive/Imocap_text/processed_tran.xlsx')
df_label=pd.read_excel('./scrap_label.xlsx')
df_label_enc = pd.read_excel('./scrap_label_enc.xlsx')
df_text=pd.read_excel('./scrap_text.xlsx')
# text=df_text.loc[df_text['sessionID']==filename]

In [13]:
docs=[]
# for text,label in zip(df['text'],df['label']):
#   if(label!=-1):
#     docs.append({'text':text,'label':label})
# print(len(docs))

needed_labels = ['neu', 'sad' , 'hap' , 'ang']

for sessionid, label in zip(df_label['sessionID'] ,df_label['label']):
#   print(label)
  if label not in needed_labels:
    continue 
  text = df_text[df_text['sessionID']==sessionid]['text']
  if len(text.values) == 0:
    continue
  else:
    text = text.values[0]
  label_enc = needed_labels.index(label)
#   print(label_enc)
  docs.append({'text': text , 'label' : label_enc})
print(len(docs))

2233


In [14]:
import random
random.shuffle(docs)
random.shuffle(docs)
random.shuffle(docs)
total_length=len(docs)
train_length=int(.9*total_length)
train_list=docs[0:train_length]
test_list=docs[train_length:]
print('no of items for train ',len(train_list))
print('no of items for test ',len(test_list))

no of items for train  2009
no of items for test  224


In [15]:
def weight_init(m):
    if isinstance(m, torch.nn.Linear):
        print('init of linear is done')
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None: 
            torch.nn.init.xavier_uniform_(m.bias)

In [16]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch import optim as opt
 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,   
    output_attentions = False,
    output_hidden_states = False, 
)
print(model)
params = list(model.named_parameters())
optimizer = opt.AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
NUM_EPOCHS=4


writer = SummaryWriter(log_dir='./content/')
total_steps = len(train_list) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [17]:
import random
total_steps = 1
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



In [ ]:
NUM_EPOCHS=4
model.to('cuda')
for epoch in range(NUM_EPOCHS):
  print(epoch)
  model.train()
  random.shuffle(train_list)
  for every_trainlist in train_list:
    label1=every_trainlist['label']
#     print(label1.values[0])
    
    label1=torch.tensor([label1])
    text=every_trainlist['text']
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    model.zero_grad()
    input_ids = input_ids.to('cuda')
    label1=label1.to('cuda')
    resp = model(input_ids,labels=label1)
    loss = resp.loss
    logits = resp.logits
    print('loss.item()',loss.item())
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    _, preds = torch.max(logits, 1)
    #print('preds',preds)
    accuracy = torch.sum(preds == label1)
    #print('accuracy.item()',accuracy.item())
    if total_steps % 10 == 0:
      with torch.no_grad():
        _, preds = torch.max(logits, 1)
        accuracy = torch.sum(preds == label1)
        writer.add_scalar('training loss', loss.item(), total_steps)
        writer.add_scalar('training accuracy', accuracy.item(), total_steps)                     
    total_steps+=1

0
loss.item() 1.5180675983428955
loss.item() 1.6960434913635254
loss.item() 1.4672878980636597
loss.item() 1.1132771968841553
loss.item() 1.458505392074585
loss.item() 1.311154842376709
loss.item() 1.3254481554031372
loss.item() 1.555227279663086
loss.item() 1.3052978515625
loss.item() 1.473909616470337
loss.item() 1.7746150493621826
loss.item() 1.198905348777771
loss.item() 1.3723514080047607
loss.item() 1.0652060508728027
loss.item() 1.333552360534668
loss.item() 1.1576554775238037
loss.item() 1.0280838012695312
loss.item() 0.9105785489082336
loss.item() 1.0292810201644897
loss.item() 1.028735637664795
loss.item() 1.0603134632110596
loss.item() 0.9687732458114624
loss.item() 1.9398401975631714
loss.item() 0.9602447152137756
loss.item() 1.666348934173584
loss.item() 1.9798563718795776
loss.item() 1.256631851196289
loss.item() 1.4738997220993042
loss.item() 1.000288724899292
loss.item() 0.8869293928146362
loss.item() 1.5627415180206299
loss.item() 0.9999203681945801
loss.item() 0.88968

loss.item() 2.9835379123687744
loss.item() 1.1154541969299316
loss.item() 0.19388258457183838
loss.item() 2.2774455547332764
loss.item() 0.8001189827919006
loss.item() 0.7650916576385498
loss.item() 2.7171437740325928
loss.item() 0.5145347118377686
loss.item() 1.6538469791412354
loss.item() 0.7478351593017578
loss.item() 1.014944314956665
loss.item() 0.18983310461044312
loss.item() 0.4335483908653259
loss.item() 0.413173109292984
loss.item() 0.3433888554573059
loss.item() 0.9814133048057556
loss.item() 1.1146705150604248
loss.item() 0.2558392882347107
loss.item() 2.8117949962615967
loss.item() 0.23748889565467834
loss.item() 1.9371395111083984
loss.item() 0.06901531666517258
loss.item() 0.1005709171295166
loss.item() 4.0834503173828125
loss.item() 0.14723612368106842
loss.item() 0.2999609410762787
loss.item() 0.04903270676732063
loss.item() 0.6637768745422363
loss.item() 1.912926197052002
loss.item() 4.0494303703308105
loss.item() 0.05037230625748634
loss.item() 3.0566272735595703
loss

loss.item() 0.5244084000587463
loss.item() 3.5251312255859375
loss.item() 1.776234745979309
loss.item() 2.4078798294067383
loss.item() 2.485996723175049
loss.item() 0.02162916585803032
loss.item() 0.0536528006196022
loss.item() 2.6693108081817627
loss.item() 0.24786575138568878
loss.item() 1.6675350666046143
loss.item() 1.5475702285766602
loss.item() 2.19527006149292
loss.item() 0.46992480754852295
loss.item() 0.09576569497585297
loss.item() 0.6141061782836914
loss.item() 1.5514127016067505
loss.item() 1.7204020023345947
loss.item() 1.5953725576400757
loss.item() 1.7443753480911255
loss.item() 0.462766170501709
loss.item() 1.7442442178726196
loss.item() 4.9651408195495605
loss.item() 1.4579887390136719
loss.item() 0.08449579030275345
loss.item() 0.22497178614139557
loss.item() 0.018070822581648827
loss.item() 2.4484148025512695
loss.item() 1.5384700298309326
loss.item() 0.014707712456583977
loss.item() 1.570853590965271
loss.item() 0.023736946284770966
loss.item() 0.3528618812561035
lo

loss.item() 0.02362343668937683
loss.item() 0.009095190092921257
loss.item() 2.334909677505493
loss.item() 0.012248064391314983
loss.item() 5.643915176391602
loss.item() 0.011629525572061539
loss.item() 1.3194199800491333
loss.item() 0.010023378767073154
loss.item() 0.04241279140114784
loss.item() 4.64542293548584
loss.item() 0.008221828378736973
loss.item() 0.017918027937412262
loss.item() 5.2859015464782715
loss.item() 4.768674850463867
loss.item() 0.7462773323059082
loss.item() 5.430241107940674
loss.item() 4.359662055969238
loss.item() 0.529632568359375
loss.item() 1.0584715604782104
loss.item() 0.0346662700176239
loss.item() 1.847086787223816
loss.item() 3.6415631771087646
loss.item() 4.082722187042236
loss.item() 4.0702385902404785
loss.item() 0.3917771577835083
loss.item() 2.6271259784698486
loss.item() 0.14454609155654907
loss.item() 0.14139851927757263
loss.item() 2.142108678817749
loss.item() 0.3052491545677185
loss.item() 0.10540701448917389
loss.item() 2.7488551139831543
lo

loss.item() 0.006965757813304663
loss.item() 0.011137695983052254
loss.item() 0.020257603377103806
loss.item() 5.171016216278076
loss.item() 0.03317756950855255
loss.item() 0.6083855032920837
loss.item() 0.02805282548069954
loss.item() 0.030715707689523697
loss.item() 0.13608258962631226
loss.item() 0.5401042103767395
loss.item() 6.395422458648682
loss.item() 0.010947405360639095
loss.item() 0.8731563091278076
loss.item() 0.013512340374290943
loss.item() 4.401236534118652
loss.item() 1.2388756275177002
loss.item() 5.445217132568359
loss.item() 0.013713902793824673
loss.item() 0.015353113412857056
loss.item() 0.019373947754502296
loss.item() 0.008912780322134495
loss.item() 0.014077058993279934
loss.item() 4.9627227783203125
loss.item() 4.5180277824401855
loss.item() 6.167079925537109
loss.item() 0.032035719603300095
loss.item() 0.009434637613594532
loss.item() 0.10493636131286621
loss.item() 0.00866274069994688
loss.item() 0.5933987498283386
loss.item() 3.7429325580596924
loss.item() 0

...

loss.item() 4.913860321044922
loss.item() 0.7409588694572449
loss.item() 0.003868597326800227
loss.item() 0.006663483567535877
loss.item() 0.0030540036968886852
loss.item() 0.00455627404153347
loss.item() 0.0033889738842844963
loss.item() 0.5620050430297852
loss.item() 6.009966850280762
loss.item() 1.9715346097946167
loss.item() 0.0015243588713929057
loss.item() 0.030652817338705063
loss.item() 0.0027608871459960938
loss.item() 0.0035151378251612186
loss.item() 0.0038402159698307514
loss.item() 0.004254101309925318
loss.item() 0.0035139499232172966
loss.item() 0.0031147319823503494
loss.item() 0.004665441811084747
loss.item() 0.0024833811912685633
loss.item() 0.0014836505288258195
loss.item() 3.4404115676879883
loss.item() 0.22763213515281677
loss.item() 0.003981521353125572
loss.item() 0.0011570908827707171
loss.item() 0.026394380256533623
loss.item() 0.0031608403660357
loss.item() 0.0020494903437793255
loss.item() 0.0022730010095983744
loss.item() 0.0026320121251046658
loss.item() 4.

loss.item() 0.048086196184158325
loss.item() 2.126523971557617
loss.item() 0.004908060654997826
loss.item() 4.121162414550781
loss.item() 0.0006756883230991662
loss.item() 6.32839822769165
loss.item() 0.005696728825569153
loss.item() 0.0031243578996509314
loss.item() 0.0041077290661633015
loss.item() 0.0055651492439210415
loss.item() 0.007290071342140436
loss.item() 0.0037836881820112467
loss.item() 0.0022250674664974213
loss.item() 0.0007526425761170685
loss.item() 0.0019991665612906218
loss.item() 0.006580824498087168
loss.item() 0.004144175909459591
loss.item() 0.00563378632068634
loss.item() 0.004687392618507147
loss.item() 0.004801885224878788
loss.item() 0.000719645875506103
loss.item() 0.059836577624082565
loss.item() 0.008499400690197945
loss.item() 0.006439175456762314
loss.item() 7.876617908477783
loss.item() 0.002100881887599826
loss.item() 0.001555662602186203
loss.item() 0.001540903584100306
loss.item() 0.005335614550858736
loss.item() 0.002923621330410242
loss.item() 0.00

loss.item() 2.1711418628692627
loss.item() 0.0027255788445472717
loss.item() 0.0016429034294560552
loss.item() 0.001879355520941317
loss.item() 5.562355995178223
loss.item() 0.0056432695128023624
loss.item() 0.008122507482767105
loss.item() 0.002952028764411807
loss.item() 5.876110553741455
loss.item() 0.0022291115019470453
loss.item() 0.004097638186067343
loss.item() 0.004063326865434647
loss.item() 5.661175727844238
loss.item() 0.0034779557026922703
loss.item() 4.521957874298096
loss.item() 0.0034794998355209827
loss.item() 0.01264412421733141
loss.item() 0.003380182199180126
loss.item() 0.00906034093350172
loss.item() 0.0008650851668789983
loss.item() 5.249782562255859
loss.item() 0.017972709611058235
loss.item() 6.685421943664551
loss.item() 0.0008111285860650241
loss.item() 4.45992374420166
loss.item() 0.45073384046554565
loss.item() 0.0036199060268700123
loss.item() 0.0034176057670265436
loss.item() 0.0031453920528292656
loss.item() 0.004957288969308138
loss.item() 0.003492923686

loss.item() 0.002092554699629545
loss.item() 0.013624298386275768
loss.item() 0.0014457975048571825
loss.item() 0.005459636449813843
loss.item() 0.009075816720724106
loss.item() 0.04467974603176117
loss.item() 0.010684546083211899
loss.item() 0.011887424625456333
loss.item() 0.0014826982514932752
loss.item() 0.005515713710337877
loss.item() 0.09546320885419846
loss.item() 0.0019214755157008767
loss.item() 3.8236429691314697
loss.item() 0.0009687021956779063
loss.item() 4.255970478057861
loss.item() 0.066241554915905
loss.item() 0.10978573560714722
loss.item() 0.0018255725735798478
loss.item() 3.876255750656128
loss.item() 0.10379131138324738
loss.item() 0.0011177966371178627
loss.item() 0.07626047730445862
loss.item() 0.005571788176894188
loss.item() 0.0014083001296967268
loss.item() 0.0031947072129696608
loss.item() 0.00508740684017539
loss.item() 5.664978504180908
loss.item() 0.012337321415543556
loss.item() 0.001513408264145255
loss.item() 2.2774219512939453
loss.item() 0.2122674733

loss.item() 0.004864761605858803
loss.item() 0.0013517538318410516
loss.item() 0.006471036467701197
loss.item() 0.002975800074636936
loss.item() 0.004948511254042387
loss.item() 5.118928909301758
loss.item() 0.0013808010844513774
loss.item() 0.1443149745464325
loss.item() 0.05483569949865341
loss.item() 0.012119817547500134
loss.item() 0.010903540998697281
loss.item() 0.37428805232048035
loss.item() 0.0009133219718933105
loss.item() 3.2051639556884766
loss.item() 0.01198036503046751
loss.item() 0.0030761088710278273
loss.item() 6.3620195388793945
loss.item() 0.006344059947878122


In [9]:
y_actu=[]
y_pred=[]
model.to('cpu')
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
total_count = 0
correct_preds = 0
total_loss = 0
for every_test_list in test_list:
    label1=every_test_list['label']
    label1=torch.tensor([label1])
    text=every_test_list['text']
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    with torch.no_grad():
      resp = model(input_ids,labels=label1)
      loss = resp.loss
      total_loss += loss.item()
      output = resp.logits
#       print(output)
      _, preds = torch.max(output, 1)
#       print(label1)
#       print(preds)
      total_count +=1
      if preds == label1 :
        correct_preds +=1
      y_actu.append(label1.numpy()[0])
      y_pred.append(preds.numpy()[0])
      

In [10]:
accuracy = correct_preds *1.0 / total_count 
print(accuracy)

0.15625


***Confusion Matrics***

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actu, y_pred)

array([[78, 11,  6,  2],
       [18, 34,  0,  2],
       [12,  3, 14,  3],
       [ 8,  1,  3, 29]])

***Save the Models***

In [30]:
torch.save(model, './content/savedModel/model_text.pt')

In [31]:
model=torch.load('./content/savedModel/model_text.pt')